In [1]:
# 2025/7/11
# zhangzhong
# https://huggingface.co/docs/transformers/tokenizer_summary

In [ ]:
# tokenizing a text is splitting it into words or subwords,
#  which then are converted to ids through a look-up table

#  Byte-Pair Encoding (BPE), WordPiece, and SentencePiece,

In [ ]:
# Word Piece
# Space and punctuation tokenization and rule-based tokenization are both examples of word tokenization
# which is loosely defined as splitting sentences into words
# pros: While it’s the most intuitive way to split texts into smaller chunks
# cons: usually generates a very big vocabulary (the set of all unique words and tokens used), ex. Transformer XL: vocab size: 267,735
#.      Such a big vocabulary size forces the model to have an enormous embedding matrix as the input and output layer, which causes both an increased memory and time complexity.
str = "Don't you love 🤗 Transformers? We sure do."
tokens = ["Do", "n't", "you", "love", "🤗", "Transformers", "?", "We", "sure", "do", "."]

# Character Piece
# While character tokenization is very simple and would greatly reduce memory and time complexity it makes it much harder for the model to learn meaningful input representations.

In [ ]:
# subword tokenization
# hybrid between word-level and character-level tokenization

# Subword tokenization algorithms rely on the principle that 
# frequently used words should not be split into smaller subwords, 
# but rare words should be decomposed into meaningful subwords.

# annoyingly -> annoying, ly
# This is especially useful in agglutinative languages(黏着语系) such as Turkish, where you can form (almost) arbitrarily long complex words by stringing together subwords.

# Note that all of those tokenization algorithms rely on some form of training which is usually done on the corpus the corresponding model will be trained on.

# In addition, subword tokenization enables the model to process words it has [never seen before], by decomposing them into known subwords
# some example
str = "I have a new GPU!"
# 1. lowercase the text
# 2. for unseen word gpu, split to gp and u
# # "##" means that the rest of the token should be attached to the previous one, without space (for decoding or reversal of the tokenization).
tokens = ["i", "have", "a", "new", "gp", "##u", "!"]

In [ ]:
# BPE, Byte-Pair Encoding
# 这一段的解释非常精彩，一下子就懂了 https://huggingface.co/docs/transformers/tokenizer_summary#byte-pair-encoding-bpe

# 1. BPE relies on a pre-tokenizer that splits the training data into words.
#   - space tokenization, GPT2, RoBERTa
#.  - rule-based tokenization: Moses, SpaCy, ftfy
# after pre-tokenization, a set of unique words has been created, and the frequency of each word is counted.

# 2. merge 
#   - use all the symbols as a base vocabulary
#.  - learns merge rules to form a new symbol from two symbols of the base vocabulary
#.  - until the vocabulary has attain the predefined desired vocabulary size


In [ ]:
# GPT2: Byte-level BPE, OpenAI 就用这个 tiktoken
# The base vocabulary of GPT-2 consists of:
# 	•	All 256 possible byte values (from 0x00 to 0xFF) — this is known as the byte-level encoding, and it’s the core idea behind GPT-2’s tokenizer.
# 	•	These 256 bytes represent every possible character in any UTF-8 encoded text, including letters, punctuation, emojis, symbols, etc., ensuring complete coverage and eliminating the need for an <unk> token.

In [ ]:
# Word Piece
# https://huggingface.co/docs/transformers/tokenizer_summary#wordpiece

# Intuitively, WordPiece is slightly different to BPE in that it evaluates what it loses by merging two symbols to ensure it’s worth it.


In [ ]:
# Sentence Piece
# https://huggingface.co/docs/transformers/tokenizer_summary#sentencepiece

# BPE and WordPiece have the same problem:
# It is assumed that the input text uses spaces to separate words, but like chinese, which do not.
# it has two solution:
#  1. use a specific pre-tokenizer for chinese, such as XLM
#. 2. SentencePiece, treats the input as a raw input stream, thus including the space in the set of characters to use.
#.    It then uses the BPE or unigram algorithm to construct the appropriate vocabulary.
#.    其实就是把空格加入到字典中呗。在字典中用下划线 _ 来表示空格。

# example:
str = "Don't you love 🤗 Transformers? We sure do."
tokens = ["▁Don", "'", "t", "▁you", "▁love", "▁", "🤗", "▁", "Transform", "ers", "?", "▁We", "▁sure", "▁do", "."]